In [1]:
import tensorflow as tf
import numpy as np
import IPython.display as display

In [3]:
tf.__version__

'2.8.0'

### tf.train.Example

#### Data Types for tf.train.Example

In [12]:
# The following functions can be used to convert a value to a type compatible
# with tf.train.Example.

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns a int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [16]:
print(_bytes_feature(b'test_string'))
print(_bytes_feature(u'test_bytes'.encode('utf-8')))

print(_float_feature(np.exp(1)))

print(_int64_feature(True))
print(_int64_feature(1))

bytes_list {
  value: "test_string"
}

bytes_list {
  value: "test_bytes"
}

float_list {
  value: 2.7182817459106445
}

int64_list {
  value: 1
}

int64_list {
  value: 1
}



In [22]:
feature = _float_feature(np.exp(1))
feature1 = _float_feature(np.exp(1.1))
print(feature, '\n', type(feature))
print(feature.SerializeToString())
print(feature1.SerializeToString())

float_list {
  value: 2.7182817459106445
}
 
 <class 'tensorflow.core.example.feature_pb2.Feature'>
b'\x12\x06\n\x04T\xf8-@'
b'\x12\x06\n\x04BD@@'


#### Creating a tf.train.